In [4]:
# need 'tables' package to read h5 files
#!conda install -c ska tables
!pip install tables
!pip install hdf5plugin
!pip install scanpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 15.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.5 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 4.5 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8048 sha256=29376ffacaba8b7073f103be80f3e07c58e980c65b222144631c4f2adf4373b5
  Stored in directory: /root/.cache/pip/wheels/bd/ad/14/6a42359351a18337a8683854cfbba99dd782271f2d1767f87f
Successfully built session-info


In [5]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import h5py, hdf5plugin
import scanpy as sc
import anndata as ad

from scipy.sparse import csr_matrix, vstack
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
import gc, random

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/open-problems-multimodal/sample_submission.csv
/kaggle/input/open-problems-multimodal/train_cite_targets.h5
/kaggle/input/open-problems-multimodal/metadata_cite_day_2_donor_27678.csv
/kaggle/input/open-problems-multimodal/test_multi_inputs.h5
/kaggle/input/open-problems-multimodal/evaluation_ids.csv
/kaggle/input/open-problems-multimodal/train_cite_inputs.h5
/kaggle/input/open-problems-multimodal/train_multi_targets.h5
/kaggle/input/open-problems-multimodal/train_multi_inputs.h5
/kaggle/input/open-problems-multimodal/metadata.csv
/kaggle/input/open-problems-multimodal/test_cite_inputs_day_2_donor_27678.h5
/kaggle/input/open-problems-multimodal/test_cite_inputs.h5


In [6]:
# score function
# pearson correlation between predicted and actual values for each sample, 
# averaged over total samples

def corrscore(ypred, yactual):
    # numpy corrcoef uses array-like
    if type(ypred) == pd.DataFrame:
        ypred = ypred.values
    if type(yactual) == pd.DataFrame:
        yactual = yactual.values
    
    # dimensions must match
    if ypred.shape == yactual.shape:
        corrsum = 0
        for sample in range(len(yactual)):
            # get 2x2 correlation matrix, select off diagonal coeff, sum 
            corrsum += np.corrcoef(ypred[sample], yactual[sample])[0,1]
    else:
        print('ypred.shape != yactual.shape')
            
    return corrsum/len(yactual)
    

In [7]:
# file directory
dir = '../input/open-problems-multimodal/'

# file paths
metadata = dir + 'metadata.csv'
test_cite_inputs = dir + 'test_cite_inputs.h5'
test_multi_inputs = dir + 'test_multi_inputs.h5'

train_cite_inputs = dir + 'train_cite_inputs.h5'
train_cite_targets = dir + 'train_cite_targets.h5'
train_multi_inputs = dir + 'train_multi_inputs.h5'
train_multi_targets = dir + 'train_multi_targets.h5'

In [8]:
# metadata file
metadf = pd.read_csv(metadata)
print(metadf.shape)
metadf.set_index('cell_id', drop=True, inplace=True)
metadf.head()

(281528, 5)


,day,donor,cell_type,technology
cell_id,,,,
c2150f55becb,2,27678,HSC,citeseq
65b7edf8a4da,2,27678,HSC,citeseq
c1b26cb1057b,2,27678,EryP,citeseq
917168fa6f83,2,27678,NeuP,citeseq
2b29feeca86d,2,27678,EryP,citeseq


In [9]:
# cite inputs - cell id x gene expression level
# read h5 file
with h5py.File(train_cite_inputs, 'r') as f:
    # print(f.keys())
    dataset = f['train_cite_inputs']
    # dataset.keys()

    # print(dataset['axis0'][0:5].astype(str)) # gene names --> features
    # print(dataset['axis1'][0:5]) # cell ids --> samples
    # print(dataset['block0_items'][0:5]) # gene names again
    # print(dataset['block0_values'][0:5]) # expression data

    # put data into anndata object for use with scanpy
    values = csr_matrix(dataset['block0_values'][:]) # sparse matrix to save space
    features = dataset['axis0'][:].astype(str) # gene features in 'ensembleid_genename' format
    samples = dataset['axis1'][:].astype(str) # cell ids

adata = ad.AnnData(
    X=values,
    obs=metadf.loc[samples],
    var=pd.DataFrame(index=features)
)
adata

AnnData object with n_obs × n_vars = 70988 × 22050
    obs: 'day', 'donor', 'cell_type', 'technology'

In [10]:
del values, features, samples
gc.collect()

46

In [11]:
print(adata)
print(adata.obs['day'].value_counts())
print(adata.obs['technology'].value_counts())
adata.obs['donor'].value_counts()

AnnData object with n_obs × n_vars = 70988 × 22050
    obs: 'day', 'donor', 'cell_type', 'technology'
4    28145
2    21942
3    20901
Name: day, dtype: int64
citeseq    70988
Name: technology, dtype: int64


31800    24803
32606    23986
13176    22199
Name: donor, dtype: int64

In [12]:
# load target data
targetf = h5py.File(train_cite_targets, 'r')
targetdata = targetf['train_cite_targets']

features = targetdata['axis0'][:].astype(str) # protein features
samples = targetdata['axis1'][:].astype(str) # cell ids
values = targetdata['block0_values'][:] # normalized protein expression values

targetdf = pd.DataFrame(values, index=samples, columns=features)
targetdf.head()

,CD86,CD274,CD270,CD155,CD112,CD47,CD48,CD40,CD154,CD52,...,CD94,CD162,CD85j,CD23,CD328,HLA-E,CD82,CD101,CD88,CD224
45006fe3e4c8,1.167804,0.622530,0.106959,0.324989,3.331674,6.426002,1.480766,-0.728392,-0.468851,-0.073285,...,-0.448390,3.220174,-0.533004,0.674956,-0.006187,0.682148,1.398105,0.414292,1.780314,0.548070
d02759a80ba2,0.818970,0.506009,1.078682,6.848758,3.524885,5.279456,4.930438,2.069372,0.333652,-0.468088,...,0.323613,8.407108,0.131301,0.047607,-0.243628,0.547864,1.832587,0.982308,2.736507,2.184063
c016c6b0efa5,-0.356703,-0.422261,-0.824493,1.137495,0.518924,7.221962,-0.375034,1.738071,0.142919,-0.971460,...,1.348692,4.888579,-0.279483,-0.131097,-0.177604,-0.689188,9.013709,-1.182975,3.958148,2.868600
ba7f733a4f75,-1.201507,0.149115,2.022468,6.021595,7.258670,2.792436,21.708519,-0.137913,1.649969,-0.754680,...,1.504426,12.391979,0.511394,0.587863,-0.752638,1.714851,3.893782,1.799661,1.537249,4.407671
fbcf2443ffb2,-0.100404,0.697461,0.625836,-0.298404,1.369898,3.254521,-1.659380,0.643531,0.902710,1.291877,...,0.777023,6.496499,0.279898,-0.841950,-0.869419,0.675092,5.259685,-0.835379,9.631781,1.765445


In [9]:
# # normalization caused some negative expression values in target
# # probably should change negative values to zero, but competition target won't have that modification
# targetdf.where(targetdf >= 0, other=0, inplace=True)
# targetdf.head()

,CD86,CD274,CD270,CD155,CD112,CD47,CD48,CD40,CD154,CD52,...,CD94,CD162,CD85j,CD23,CD328,HLA-E,CD82,CD101,CD88,CD224
45006fe3e4c8,1.167804,0.622530,0.106959,0.324989,3.331674,6.426002,1.480766,0.000000,0.000000,0.000000,...,0.000000,3.220174,0.000000,0.674956,0.0,0.682148,1.398105,0.414292,1.780314,0.548070
d02759a80ba2,0.818970,0.506009,1.078682,6.848758,3.524885,5.279456,4.930438,2.069372,0.333652,0.000000,...,0.323613,8.407108,0.131301,0.047607,0.0,0.547864,1.832587,0.982308,2.736507,2.184063
c016c6b0efa5,0.000000,0.000000,0.000000,1.137495,0.518924,7.221962,0.000000,1.738071,0.142919,0.000000,...,1.348692,4.888579,0.000000,0.000000,0.0,0.000000,9.013709,0.000000,3.958148,2.868600
ba7f733a4f75,0.000000,0.149115,2.022468,6.021595,7.258670,2.792436,21.708519,0.000000,1.649969,0.000000,...,1.504426,12.391979,0.511394,0.587863,0.0,1.714851,3.893782,1.799661,1.537249,4.407671
fbcf2443ffb2,0.000000,0.697461,0.625836,0.000000,1.369898,3.254521,0.000000,0.643531,0.902710,1.291877,...,0.777023,6.496499,0.279898,0.000000,0.0,0.675092,5.259685,0.000000,9.631781,1.765445


In [13]:
# filter out cells with less than 200 genes expressed, 
# filter out genes expressed in fewer than 7 cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=7)
# sc.pp.filter_genes(adata, min_cells=70)
gc.collect()
adata.shape


(70988, 21601)

In [26]:
# reduce features
svd = TruncatedSVD(n_components=15, n_iter=7, random_state=765)
adatasvd = svd.fit_transform(adata.X)
print(svd.explained_variance_ratio_)
print(f'shape {adata.shape} -> {adatasvd.shape}')
del adatasvd
gc.collect()

[0.02765413 0.02171798 0.01394943 0.01052559 0.00827429 0.00524452
 0.00394503 0.00325971 0.00299689 0.0022196  0.00209268 0.00178203
 0.00156881 0.0013976  0.00127581]
(70988, 20178) -> (70988, 15)


In [15]:
# group kfold
# public test set contains data from donor not in train data
# private test set contains data from day not in train data
gc.collect()

scorelist = []
scorelist2 = []
mse1list = []
mse2list = []

gkf = GroupKFold(n_splits=3)

for trainidx, testidx in gkf.split(adata.X, groups=adata.obs['day']):
    trainx, trainy = adata.X[trainidx], targetdf.reset_index(drop=True).loc[trainidx]
    testx, testy = adata.X[testidx], targetdf.reset_index(drop=True).loc[testidx]
    
    print(f"test group is day: {adata.obs['day'][testidx[0]]}")
    
    # feature reduction
    svd = TruncatedSVD(n_components=50, n_iter=7, random_state=765)
    trainx_svd = pd.DataFrame(svd.fit_transform(trainx))
    testx_svd = pd.DataFrame(svd.transform(testx))
    
    trainx_svd.columns = trainx_svd.columns.astype(str)
    testx_svd.columns = testx_svd.columns.astype(str)
    
    
    km = KMeans(n_clusters=10, random_state=765)
    trainx_svd['cluster'] = km.fit_predict(trainx_svd)
    testx_svd['cluster'] = km.predict(testx_svd)

    
    # add 'day' feature from metadata
    trainx_svd['day'] = list(adata.obs['day'].reset_index(drop=True).loc[trainidx])
    testx_svd['day'] = list(adata.obs['day'].reset_index(drop=True).loc[testidx])
    
    
    # ridge regression model
    ridgepipe = Pipeline([
        ('rscaler', RobustScaler()),
        ('ridge', Ridge())
    ])
    
    ridgepipe.fit(trainx_svd, trainy)
    predy = ridgepipe.predict(testx_svd)
    
    # xgboost model
    xgb = XGBRegressor(n_estimators=50, max_depth=5, tree_method='hist')
    multixgb = MultiOutputRegressor(xgb)

    multixgb.fit(trainx_svd, trainy)
    predy2 = multixgb.predict(testx_svd)

    # ensemble prediction
    predy12 = 0.35*(predy) + 0.65*(predy2)
    predy122 = 0.3*(predy) + 0.7*(predy2)

    # store scores
    scorelist.append(corrscore(predy12, testy))
    scorelist2.append(corrscore(predy122, testy))
    mse1list.append(mean_squared_error(testy, predy))
    mse2list.append(mean_squared_error(testy, predy2))
    
    

print(f'mean corr score: {np.mean(scorelist)}, scores: {scorelist}')
print(f'mean corr score: {np.mean(scorelist2)}, scores: {scorelist2}')
print(f'mean mse ridge: {np.mean(mse1list)}, scores: {mse1list}')
print(f'mean mse xgb: {np.mean(mse2list)}, scores: {mse2list}')



test group is day: 4
test group is day: 2
test group is day: 3
mean corr score: 0.885590896867703, scores: [0.8800160968429539, 0.8852305437028921, 0.8915260500572632]
mean corr score: 0.8854194740939887, scores: [0.8802080236302072, 0.8850750305368912, 0.8909753681148674]
mean mse ridge: 2.9723714385988185, scores: [2.9899438708046913, 3.0179710571615583, 2.909199387830206]
mean mse xgb: 2.936772584915161, scores: [2.7853072, 2.983472, 3.0415392]


In [13]:
%%time
xgb = XGBRegressor(n_estimators=50, max_depth=5, tree_method='hist')
multixgb = MultiOutputRegressor(xgb)

multixgb.fit(trainx_svd, trainy)
predy2 = multixgb.predict(testx_svd)
print(f'corr score: {corrscore(predy2, testy)}')
multixgb.score(testx_svd, testy)

corr score: 0.8857862279864123
CPU times: user 10min 26s, sys: 1.24 s, total: 10min 28s
Wall time: 2min 42s


0.13443152752918558